In [10]:
this_notebook_name = "F-ScorePlotTest"

# folder to save notebook html
local_data_folder = r"c:\Data\SagittalSpineSegmentationStudy"
overwrite_existing_data_files = False


# All results and output will be archived with this timestamp

import datetime
save_timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
print("Save timestamp: {}".format(save_timestamp))

# Learning parameters

import numpy as np

ultrasound_size = 128
num_classes = 2
num_epochs = 500
batch_size = 128
max_learning_rate = 0.02
min_learning_rate = 0.00001
regularization_rate = 0.0001
filter_multiplier = 8
class_weights = np.array([0.1, 0.9])
learning_rate_decay = (max_learning_rate - min_learning_rate) / num_epochs

# Training data augmentation parameters

max_shift_factor = 0.12
max_rotation_angle = 10
max_zoom_factor = 1.1
min_zoom_factor = 0.8

# Evaluation parameters

acceptable_margin_mm = 1.0
mm_per_pixel = 1.0

roc_thresholds = [0.9, 0.8, 0.7, 0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1,
                  0.08, 0.06, 0.04, 0.02, 0.01,
                  0.008, 0.006, 0.004, 0.002, 0.001]

# Validation schedule - applied across all tests
validation_schedule_patient = np.array([[0]])
num_validation_rounds = len(validation_schedule_patient)

# uncomment for faster debugging
# roc_thresholds = [0.8, 0.6, 0.4, 0.2, 0.1, 0.01, 0.001]
# num_epochs = 5

Save timestamp: 2020-08-08_16-34-43


In [11]:
import os
import sys
from random import sample
from pathlib import Path

from ipywidgets import IntProgress
from IPython.display import display, HTML

import girder_client
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf

import ultrasound_batch_generator as generator
import evaluation_metrics

In [12]:
# Import aigt modules

parent_folder = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(parent_folder)

import Models.segmentation_unet as unet
import utils

In [13]:
# Creating standard folders to save data and logs

data_arrays_fullpath, notebooks_save_fullpath, results_save_fullpath, models_save_fullpath, val_data_fullpath =\
    utils.create_standard_project_folders(local_data_folder)

In [14]:
# loading patient data

girder_url = "https://pocus.cs.queensu.ca/api/v1"

from girder_api_key import girder_key

data_csv_file = "QueensSagittal.csv"

ultrasound_arrays_by_patients, segmentation_arrays_by_patients =\
    utils.load_girder_data(data_csv_file, data_arrays_fullpath, girder_url, girder_key=girder_key)

data_csv_file = "VerdureSagittal.csv"

ultrasound_arrays_by_patients2, segmentation_arrays_by_patients2 =\
    utils.load_girder_data(data_csv_file, data_arrays_fullpath, girder_url, girder_key=girder_key)

ultrasound_arrays_by_patients += ultrasound_arrays_by_patients2
segmentation_arrays_by_patients += segmentation_arrays_by_patients2

n_patients = len(ultrasound_arrays_by_patients)

for i in range(n_patients):
    print("Patient {} has {} ultrasounds and {} segmentations".format(
        i, ultrasound_arrays_by_patients[i].shape[0], segmentation_arrays_by_patients[i].shape[0]))

Patient 0 has 387 ultrasounds and 387 segmentations
Patient 1 has 360 ultrasounds and 360 segmentations
Patient 2 has 453 ultrasounds and 453 segmentations
Patient 3 has 289 ultrasounds and 289 segmentations
Patient 4 has 477 ultrasounds and 477 segmentations
Patient 5 has 446 ultrasounds and 446 segmentations
Patient 6 has 355 ultrasounds and 355 segmentations
Patient 7 has 523 ultrasounds and 523 segmentations
Patient 8 has 425 ultrasounds and 425 segmentations
Patient 9 has 502 ultrasounds and 502 segmentations
Patient 10 has 134 ultrasounds and 134 segmentations
Patient 11 has 77 ultrasounds and 77 segmentations
Patient 12 has 187 ultrasounds and 187 segmentations
Patient 13 has 79 ultrasounds and 79 segmentations
Patient 14 has 178 ultrasounds and 178 segmentations
Patient 15 has 26 ultrasounds and 26 segmentations
Patient 16 has 349 ultrasounds and 349 segmentations
Patient 17 has 378 ultrasounds and 378 segmentations
Patient 18 has 526 ultrasounds and 526 segmentations
Patient 1

In [ ]:
# Print training parameters, to archive them together with the notebook output.

time_sequence_start = datetime.datetime.now()

print("Beginning test")

f_score_list = []

for i in range(2, len(ultrasound_arrays_by_patients) + 1):
    
    ultrasound_sub_array = ultrasound_arrays_by_patients[0:i]
    segmentation_sub_array = segmentation_arrays_by_patients[0:i]
    n_patients = len(ultrasound_sub_array)

    # ROC data will be saved in these containers

    val_best_metrics    = dict()
    val_fuzzy_metrics   = dict()
    val_aurocs          = np.zeros(num_validation_rounds)
    val_best_thresholds = np.zeros(num_validation_rounds)

    # Perform validation rounds

    for val_round_index in range(num_validation_rounds):

        # Prepare data arrays

        train_ultrasound_data = np.zeros(
            [0,
             ultrasound_sub_array[0].shape[1],
             ultrasound_sub_array[0].shape[2],
             ultrasound_sub_array[0].shape[3]])

        train_segmentation_data = np.zeros(
            [0,
             segmentation_sub_array[0].shape[1],
             segmentation_sub_array[0].shape[2],
             segmentation_sub_array[0].shape[3]])

        val_ultrasound_data = np.zeros(
            [0,
             ultrasound_sub_array[0].shape[1],
             ultrasound_sub_array[0].shape[2],
             ultrasound_sub_array[0].shape[3]])

        val_segmentation_data = np.zeros(
            [0,
             segmentation_sub_array[0].shape[1],
             segmentation_sub_array[0].shape[2],
             segmentation_sub_array[0].shape[3]])

        for patient_index in range(n_patients):
            if patient_index not in validation_schedule_patient[val_round_index]:
                train_ultrasound_data = np.concatenate((train_ultrasound_data,
                                                        ultrasound_sub_array[patient_index]))
                train_segmentation_data = np.concatenate((train_segmentation_data,
                                                          segmentation_sub_array[patient_index]))
            else:
                val_ultrasound_data = np.concatenate((val_ultrasound_data,
                                                     ultrasound_sub_array[patient_index]))
                val_segmentation_data = np.concatenate((val_segmentation_data,
                                                       segmentation_sub_array[patient_index]))

        n_train = train_ultrasound_data.shape[0]
        n_val = val_ultrasound_data.shape[0]

        print("\n*** # of patients in this round: {}".format(val_round_index + 1))
        print("    Training on {} images, validating on {} images...".format(n_train, n_val))

        val_segmentation_data_onehot = tf.keras.utils.to_categorical(val_segmentation_data, num_classes)

        # Create and train model

        model = unet.segmentation_unet(ultrasound_size, num_classes, filter_multiplier, regularization_rate)

        print(tf.config.experimental.list_physical_devices('GPU'))
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(lr=max_learning_rate, decay=learning_rate_decay),
            loss=unet.weighted_categorical_crossentropy(class_weights),
            metrics=["accuracy"]
        )

        # model.summary()

        training_generator = generator.UltrasoundSegmentationBatchGenerator(
            train_ultrasound_data,
            train_segmentation_data[:, :, :, 0],
            batch_size,
            (ultrasound_size, ultrasound_size),
            max_shift_factor=max_shift_factor,
            min_zoom_factor=min_zoom_factor,
            max_zoom_factor=max_zoom_factor,
            max_rotation_angle=max_rotation_angle
        )

        training_time_start = datetime.datetime.now()

        if n_val > 0:
            training_log = model.fit_generator(
                training_generator,
                validation_data=(val_ultrasound_data, val_segmentation_data_onehot),
                epochs=num_epochs,
                verbose=0)
        else:
            training_log = model.fit_generator(training_generator, epochs=num_epochs, verbose=0)

        training_time_stop = datetime.datetime.now()

        # Pring training log

        print("  Training time: {}".format(training_time_stop-training_time_start))

        # Plot training loss and metrics

        # fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

        # axes[0].plot(training_log.history['loss'], 'bo--')
        # if n_val > 0:
        #     axes[0].plot(training_log.history['val_loss'], 'ro-')
        # axes[0].set(xlabel='Epochs (n)', ylabel='Loss')
        # if n_val > 0:
        #    axes[0].legend(['Training loss', 'Validation loss'])

        # axes[1].plot(training_log.history['accuracy'], 'bo--')
        # if n_val > 0:
        #     axes[1].plot(training_log.history['val_accuracy'], 'ro-')
        # axes[1].set(xlabel='Epochs (n)', ylabel='Accuracy')
        # if n_val > 0:
        #     axes[1].legend(['Training accuracy', 'Validation accuracy'])

        # fig.tight_layout()

        # Archive trained model with unique filename based on notebook name and timestamp

        model_file_name = this_notebook_name + "_model-" + str(i) + "_" + save_timestamp + ".h5"
        model_fullname = os.path.join(models_save_fullpath, model_file_name)
        model.save(model_fullname)

        # Predict on validation data

        if n_val > 0:
            y_pred_val  = model.predict(val_ultrasound_data)

            # Saving predictions for further evaluation

            # val_prediction_filename = save_timestamp + "_prediction_" + str(val_round_index) + ".npy"
            # val_prediction_fullname = os.path.join(val_data_fullpath, val_prediction_filename)
            # np.save(val_prediction_fullname, y_pred_val)

            # Validation results

            vali_metrics_dicts, vali_best_threshold_index, vali_area = evaluation_metrics.compute_roc(
                roc_thresholds, y_pred_val, val_segmentation_data, acceptable_margin_mm, mm_per_pixel)

            val_fuzzy_metrics[val_round_index] = evaluation_metrics.compute_evaluation_metrics(
                y_pred_val, val_segmentation_data, acceptable_margin_mm, mm_per_pixel)

            val_best_metrics[val_round_index]    = vali_metrics_dicts[vali_best_threshold_index]
            val_aurocs[val_round_index]          = vali_area
            val_best_thresholds[val_round_index] = roc_thresholds[vali_best_threshold_index]
            
            f_score_list.append(val_fuzzy_metrics[0][evaluation_metrics.FSCORE])

        # Printing total time of this validation round

        print("\nTotal round time:  {}".format(datetime.datetime.now() - training_time_start))
        print("")


time_sequence_stop = datetime.datetime.now()

print("\nTotal training time:   {}".format(time_sequence_stop - time_sequence_start))


Beginning test

*** # of patients in this round: 1
    Training on 360 images, validating on 387 images...
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
  Training time: 0:08:03.341974

Total round time:  0:08:11.379481


*** # of patients in this round: 1
    Training on 813 images, validating on 387 images...
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
  Training time: 0:22:00.834477

Total round time:  0:22:08.956756


*** # of patients in this round: 1
    Training on 1102 images, validating on 387 images...
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
  Training time: 0:29:15.164831

Total round time:  0:29:23.498545


*** # of patients in this round: 1
    Training on 1579 images, validating on 387 images...
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
  Training time: 0:43:13.653688

Total round time:  0:43:22.050264


*** # of patients in this round: 1
    Training on 2025 images, validating

In [1]:
plt.figure()
plt.ylim(-0.01, 1.01)
plt.plot(f_score_list, color='darkred', lw=2)

NameError: name 'plt' is not defined

In [ ]:
# Save notebook so all output is archived by the next cell

from IPython.display import Javascript
script = '''
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});
'''
Javascript(script)

In [ ]:
# Export HTML copy of this notebook

notebook_file_name = this_notebook_name + "_" + save_timestamp + ".html"
notebook_fullname = os.path.join(notebooks_save_fullpath, notebook_file_name)

os.system("jupyter nbconvert --to html " + this_notebook_name + " --output " + notebook_fullname)
print("Notebook saved to: {}".format(notebook_fullname))